In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from datetime import datetime


# Import API key
from api_key import congress_api_key

In [22]:
# API call to return bills from current congress (ie 118th)

exapmple_url = "https://api.congress.gov/v3/bill?api_key=[INSERT_KEY]"


query_url = f"https://api.congress.gov/v3/bill?api_key={congress_api_key}"

bill_response = requests.get(query_url)
bill_json = bill_response.json()
# bill get request example url
"https://api.congress.gov/v3/bill?api_key=[INSERT_KEY]"

'https://api.congress.gov/v3/bill?api_key=[INSERT_KEY]'

In [23]:
bill_json

{'bills': [{'congress': 118,
   'latestAction': {'actionDate': '2023-11-16',
    'text': 'Presented to President.'},
   'number': '6363',
   'originChamber': 'House',
   'originChamberCode': 'H',
   'title': 'Further Continuing Appropriations and Other Extensions Act, 2024',
   'type': 'HR',
   'updateDate': '2023-11-16',
   'updateDateIncludingText': '2023-11-16T20:43:49Z',
   'url': 'https://api.congress.gov/v3/bill/118/hr/6363?format=json'},
  {'congress': 118,
   'latestAction': {'actionDate': '2023-11-16',
    'text': 'Introduced in the Senate. Read the first time. Placed on Senate Legislative Calendar under Read the First Time.'},
   'number': '3343',
   'originChamber': 'Senate',
   'originChamberCode': 'S',
   'title': 'A bill to provide that United States citizens evacuating Israel shall not be required to reimburse the United States Government, and for other purposes.',
   'type': 'S',
   'updateDate': '2023-11-16',
   'updateDateIncludingText': '2023-11-16T09:46:34Z',
   'ur

In [4]:
# Query to return text versions of specific bill

example_url = "https://api.congress.gov/v3/bill/117/hr/3076/text?api_key=[INSERT_KEY]"
bill_no = "3075"
query_url = f"https://api.congress.gov/v3/bill/117/hr/{bill_no}/text?api_key={congress_api_key}"

texts_response = requests.get(query_url)
texts_json = texts_response.json()

In [5]:
texts_json

{'pagination': {'count': 2},
 'request': {'billNumber': '3075',
  'billType': 'hr',
  'billUrl': 'https://api.congress.gov/v3/bill/117/hr/3075?format=json',
  'congress': '117',
  'contentType': 'application/json',
  'format': 'json'},
 'textVersions': [{'date': '2022-12-30T05:00:00Z',
   'formats': [{'type': 'Formatted Text',
     'url': 'https://www.congress.gov/117/bills/hr3075/BILLS-117hr3075rh.htm'},
    {'type': 'PDF',
     'url': 'https://www.congress.gov/117/bills/hr3075/BILLS-117hr3075rh.pdf'},
    {'type': 'Formatted XML',
     'url': 'https://www.congress.gov/117/bills/hr3075/BILLS-117hr3075rh.xml'}],
   'type': 'Reported in House'},
  {'date': '2021-05-11T04:00:00Z',
   'formats': [{'type': 'Formatted Text',
     'url': 'https://www.congress.gov/117/bills/hr3075/BILLS-117hr3075ih.htm'},
    {'type': 'PDF',
     'url': 'https://www.congress.gov/117/bills/hr3075/BILLS-117hr3075ih.pdf'},
    {'type': 'Formatted XML',
     'url': 'https://www.congress.gov/117/bills/hr3075/BILLS

In [50]:
json_data = texts_json['textVersions'][0]['formats'][0]['url']
json_data

'https://www.congress.gov/115/bills/hr5/BILLS-115hr5rfs.htm'

In [51]:
# Scraping text with BeautifulSoup

import requests
from bs4 import BeautifulSoup

url =  json_data
result = requests.get(url)
content = result.text

soup = BeautifulSoup(content, 'lxml')

#print(soup)

In [52]:
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [53]:
soup_string = str(soup)
#print(soup_string)

In [54]:
print(len(soup_string))

113450


In [55]:
print(soup_string)

<html><body><pre>
[Congressional Bills 115th Congress]
[From the U.S. Government Publishing Office]
[H.R. 5 Referred in Senate (RFS)]

&lt;DOC&gt;
115th CONGRESS
  1st Session
                                 H. R. 5


_______________________________________________________________________


                   IN THE SENATE OF THE UNITED STATES

                            January 12, 2017

Received; read twice and referred to the Committee on Homeland Security 
                        and Governmental Affairs

_______________________________________________________________________

                                 AN ACT


 
 To reform the process by which Federal agencies analyze and formulate 
   new regulations and guidance documents, to clarify the nature of 
judicial review of agency interpretations, to ensure complete analysis 
    of potential impacts on small entities of rules, and for other 
                               purposes.

    Be it enacted by the Senate and House o

In [48]:
# Summarizing with T5

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Set up the summarization pipeline
tokenizer = AutoTokenizer.from_pretrained("t5-base",max_input_length = 1024, model_max_length = 1024)
model1 = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
summarizer = pipeline("summarization", model=model1, tokenizer=tokenizer)

# Define a function to generate summaries for each section
def generate_summary(text):
    # Generate a summary of the text using the summarization pipelineb
     
    summary = summarizer(text, max_length=80, min_length=25, do_sample=False)[0]['summary_text']
    return summary

In [ ]:
generate_summary(soup_string)

Token indices sequence length is longer than the specified maximum sequence length for this model (12025 > 1024). Running this sequence through the model will result in indexing errors
/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/transformers/generation/utils.py:1202: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"


In [2]:
# ChatGPT-edited code for splitting text
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Set up the summarization pipeline
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# Define a function to generate summaries for each chunk
def generate_summary(text, max_chunk_length=512, max_summary_length=80, min_summary_length=25):
    # Split the text into chunks
    chunks = [text[i:i + max_chunk_length] for i in range(0, len(text), max_chunk_length)]

    # Generate summaries for each chunk
    summaries = [summarizer(chunk, max_length=max_summary_length, min_length=min_summary_length, do_sample=False)[0]['summary_text'] for chunk in chunks]

    # Concatenate the summaries
    full_summary = ' '.join(summaries)

    return full_summary




2023-11-16 16:06:23.768316: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   return full_summary                                                                     │
│   21                                                                                             │
│   22 # Assuming soup_string is your long text                                                    │
│ ❱ 23 full_summary = generate_summary(soup_string)                                                │
│   24                                                                                             │
│   25 print(full_summary)                                                                         │
│   26                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'soup_string' is not defined

In [8]:
# Assuming soup_string is your long text
full_summary = generate_summary(soup_string)

print(full_summary)

/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/transformers/generation/utils.py:1202: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"
Your max_length is set to 80, but you input_length is only 67. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)
Your max_length is set to 80, but you input_length is only 79. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 80, but you input_length is only 76. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Your max_length is set to 80, but you input_leng

[H.R. 3075] To address seafood slavery and combat illegal, unreported, or unregulated fishing . [from the u.s. government publishing office] bill referred to committees on ways and means, transportation and infrastructure, agriculture, and energy and commerce . speaker to decide whether bill will be considered for a period to be determined . the Committee on Natura tion of such provisions as fall within the jurisdiction of the committee concerned December 30, 2022 Additional sponsors: Mr. Palazzo, Mr. Case, Ms. Norton; Mr. Lowenthal; Ms . Barragan; mr. Blumenauer; dr. Moore of Alabama; ed. committees on ways and means, transportation and infrastructure, agriculture, and energy and commerce discharged . bill committed to the Committee of the Whole House on the State of the Union and ordered to be printed . bill addresses seafood slavery and combat illegal, unreported, or unregulated fishing . bill may be cited as the Illegal Fishing and Forced Labor Prevention Act' oppressive child labo

In [11]:
# Attempt at splitting bill into sections
import pandas as pd
import re

def extract_sections(input_text):
    # Define a regular expression pattern to identify sections
    section_pattern = re.compile(r'SEC\.\s+(\d+)\.\s+(.*?)\n', re.DOTALL)

    # Find all matches in the input text
    matches = section_pattern.findall(input_text)

    # Create a DataFrame from the matches
    df = pd.DataFrame(matches, columns=['SectionNumber', 'SectionText'])

    return df




In [12]:
# trying on soup_string


result_df = extract_sections(soup_string)
print(result_df)

   SectionNumber                                        SectionText
0              2                                       DEFINITIONS.
1            101                                       DEFINITIONS.
2            102  EXPANSION OF SEAFOOD IMPORT MONITORING PROGRAM...
3            103  ENHANCEMENT OF SEAFOOD IMPORT MONITORING PROGR...
4            104  ADDITIONAL DATA REQUIREMENTS FOR SEAFOOD IMPOR...
5            105  EFFORTS TO IMPROVE DETECTION OF AT-RISK SEAFOO...
6            106                                     IMPORT AUDITS.
7            107                          INTERAGENCY COORDINATION.
8            108             AVAILABILITY OF FISHERIES INFORMATION.
9            109               REPORT ON SEAFOOD IMPORT MONITORING.
10           110                   AUTHORIZATION OF APPROPRIATIONS.
11           201    FEDERAL ACTIVITIES ON SEAFOOD SAFETY AND FRAUD.
12           202               SEAFOOD LABELING AND IDENTIFICATION.
13           203                               F

In [14]:
# Assuming result_df is the DataFrame obtained from extract_sections function
selected_text = result_df.loc[result_df['SectionNumber'] == '205', 'SectionText'].values[0]

print(selected_text)


EFFECT ON STATE LAW.


In [20]:
import pandas as pd
import re

def extract_sections(input_text):
    # Define a regular expression pattern to identify sections
    section_pattern = re.compile(r'SEC\.\s+(\d+)\.\s+(.*?)(?=\nSEC\.\s+\d+\.|$)', re.DOTALL)

    # Find all matches in the input text
    matches = section_pattern.findall(input_text)

    # Create a DataFrame from the matches
    df = pd.DataFrame(matches, columns=['SectionNumber', 'SectionText'])
    df['SectionTitle'] = df['SectionText'].apply(lambda x: x.split('.')[0].strip())

    return df






In [21]:
# Example usage

result_df = extract_sections(soup_string)
print(result_df)

   SectionNumber                                        SectionText  \
0              2  DEFINITIONS.\n\n    In this Act, the following...   
1            101  DEFINITIONS.\n\n    In this title, the followi...   
2            102  EXPANSION OF SEAFOOD IMPORT MONITORING PROGRAM...   
3            103  ENHANCEMENT OF SEAFOOD IMPORT MONITORING PROGR...   
4            104  ADDITIONAL DATA REQUIREMENTS FOR SEAFOOD IMPOR...   
5            105  EFFORTS TO IMPROVE DETECTION OF AT-RISK SEAFOO...   
6            106  IMPORT AUDITS.\n\n    (a) Audit Procedures.--T...   
7            107  INTERAGENCY COORDINATION.\n\n    The Secretary...   
8            108  AVAILABILITY OF FISHERIES INFORMATION.\n\n    ...   
9            109  REPORT ON SEAFOOD IMPORT MONITORING.\n\n    (a...   
10           110  AUTHORIZATION OF APPROPRIATIONS.\n\n    There ...   
11           201  FEDERAL ACTIVITIES ON SEAFOOD SAFETY AND FRAUD...   
12           202  SEAFOOD LABELING AND IDENTIFICATION.\n\n    (a...   
13    

In [22]:



# Specify the path where you want to save the CSV file
csv_file_path = 'soup_string_sectioned.csv'

# Save the DataFrame as a CSV file
result_df.to_csv(csv_file_path, index=False)


In [40]:
# import pandas as pd
# import re

def extract_sections(input_text):
    # Define a regular expression pattern to identify sections
    section_pattern = re.compile(r'SEC\.\s+(\d+)\.\s+(.*?)(?=\nSEC\.\s+\d+\.|$)', re.DOTALL)

    # Find all matches in the input text
    matches = section_pattern.findall(input_text)

    # Create a DataFrame from the matches
    df = pd.DataFrame(matches, columns=['SectionNumber', 'SectionText'])
    df['SectionTitle'] = df['SectionText'].apply(lambda x: x.split('.')[0].strip())

    # Exclude the title from the SectionText column
    df['SectionTextWithoutTitle'] = df.apply(lambda row: row['SectionText'].replace(row['SectionTitle'], '').strip(), axis=1)

    return df


In [24]:
result_df = extract_sections(soup_string)
print(result_df)

   SectionNumber                                        SectionText  \
0              2  DEFINITIONS.\n\n    In this Act, the following...   
1            101  DEFINITIONS.\n\n    In this title, the followi...   
2            102  EXPANSION OF SEAFOOD IMPORT MONITORING PROGRAM...   
3            103  ENHANCEMENT OF SEAFOOD IMPORT MONITORING PROGR...   
4            104  ADDITIONAL DATA REQUIREMENTS FOR SEAFOOD IMPOR...   
5            105  EFFORTS TO IMPROVE DETECTION OF AT-RISK SEAFOO...   
6            106  IMPORT AUDITS.\n\n    (a) Audit Procedures.--T...   
7            107  INTERAGENCY COORDINATION.\n\n    The Secretary...   
8            108  AVAILABILITY OF FISHERIES INFORMATION.\n\n    ...   
9            109  REPORT ON SEAFOOD IMPORT MONITORING.\n\n    (a...   
10           110  AUTHORIZATION OF APPROPRIATIONS.\n\n    There ...   
11           201  FEDERAL ACTIVITIES ON SEAFOOD SAFETY AND FRAUD...   
12           202  SEAFOOD LABELING AND IDENTIFICATION.\n\n    (a...   
13    

In [25]:
# Specify the path where you want to save the CSV file
csv_file_path = 'soup_string_sectioned_2.csv'

# Save the DataFrame as a CSV file
result_df.to_csv(csv_file_path, index=False)


In [26]:
# Assuming result_df is your DataFrame with the 'SectionTextWithoutTitle' column
selected_row_index = 20  # Change this to the index of the row you want to summarize

# Extract the text from the 'SectionTextWithoutTitle' column for the selected row
selected_text = result_df.loc[selected_row_index, 'SectionTextWithoutTitle']

# Run generate_summary on the selected text
summary = generate_summary(selected_text)

# Print the summary
print(summary)


Your max_length is set to 80, but you input_length is only 49. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)


not later than 1 year after the date of enactment of this Act, the Secretary shall promulgate regulations implementing this title.


In [33]:
selected_row_index = 8  # Change this to the index of the row you want to summarize

# Extract the text from the 'SectionTextWithoutTitle' column for the selected row
selected_text = result_df.loc[selected_row_index, 'SectionTextWithoutTitle']

# Run generate_summary on the selected text
summary = generate_summary(selected_text)

# Print the summary
print(summary)

the Magnuson-Stevens Fishery Conservation and Management Act (16 U.S.C. 1881a(b)(1)) is amended by striking or'' after the semicolon at the end of subparagraph (G) and by striking the period at the . end of paragraph (H) and inserting ; or'' . r or with respect to-- (i) the Seafood Import Monitoring Program; '(ii) 'the Antarctic Marine Living Resources Program;' ''iii'' the Tuna Tracking and Verification Program'' . 'iv) the Atlantic Highly Migratory Species International Trade Program' produced by Child Labor or Forced Labor in accordance with the Trafficking Victims Protection Act of 2000 (22 U.S.C. 7101 et seq.) regulations authorized under section 307 of the Tariff Act of 1930 (19 U.S.C. 1307); and (viii) the taking and related acts in commercial fishing operations . the Secretary shall issue regulations implementing the amendments in this section . not later than 1 year after the date of enactment of this Act .


In [34]:
result_df['SectionTextWithoutTitle'][8]

".\n\n    (a) In General.--Section 402(b)(1) of the Magnuson-Stevens Fishery \nConservation and Management Act (16 U.S.C. 1881a(b)(1)) is amended by \nstriking ``or'' after the semicolon at the end of subparagraph (G), by \nstriking the period at the end of subparagraph (H) and inserting ``; \nor'' , and by adding at the end the following:\n                    ``(I) to Federal agencies responsible for screening \n                of imported seafood and for the purpose of carrying out \n                the duties under or with respect to--\n                            ``(i) the Seafood Import Monitoring \n                        Program;\n                            ``(ii) the Antarctic Marine Living \n                        Resources Program;\n                            ``(iii) the Tuna Tracking and Verification \n                        Program;\n                            ``(iv) the Atlantic Highly Migratory \n                        Species International Trade Program;\n         

## Testing on another large bill available in downloaded data source 

In [36]:
# Trying again on another bill: HR5 of congress 115 
#example_url = "https://api.congress.gov/v3/bill/117/hr/3076/text?api_key=[INSERT_KEY]"
bill_no = "5"
query_url = f"https://api.congress.gov/v3/bill/115/hr/{bill_no}/text?api_key={congress_api_key}"

texts_response = requests.get(query_url)
texts_json = texts_response.json()

In [37]:
texts_json

{'pagination': {'count': 3},
 'request': {'billNumber': '5',
  'billType': 'hr',
  'billUrl': 'https://api.congress.gov/v3/bill/115/hr/5?format=json',
  'congress': '115',
  'contentType': 'application/json',
  'format': 'json'},
 'textVersions': [{'date': '2017-01-12T05:00:00Z',
   'formats': [{'type': 'Formatted Text',
     'url': 'https://www.congress.gov/115/bills/hr5/BILLS-115hr5rfs.htm'},
    {'type': 'PDF',
     'url': 'https://www.congress.gov/115/bills/hr5/BILLS-115hr5rfs.pdf'},
    {'type': 'Formatted XML',
     'url': 'https://www.congress.gov/115/bills/hr5/BILLS-115hr5rfs.xml'}],
   'type': 'Referred in Senate'},
  {'date': '2017-01-11T05:00:00Z',
   'formats': [{'type': 'Formatted Text',
     'url': 'https://www.congress.gov/115/bills/hr5/BILLS-115hr5eh.htm'},
    {'type': 'PDF',
     'url': 'https://www.congress.gov/115/bills/hr5/BILLS-115hr5eh.pdf'},
    {'type': 'Formatted XML',
     'url': 'https://www.congress.gov/115/bills/hr5/BILLS-115hr5eh.xml'}],
   'type': 'Engro

In [ ]:
# Scraping text with BeautifulSoup

import requests
from bs4 import BeautifulSoup

url =  'https://www.congress.gov/117/bills/hr3075/BILLS-117hr3075rh.htm'
result = requests.get(url)
content = result.text

soup = BeautifulSoup(content, 'lxml')

print(soup)

In [44]:
# trying again with BeautifulSoup

#import requests
#from bs4 import BeautifulSoup

url =  'https://www.congress.gov/115/bills/hr5/BILLS-115hr5rfs.htm'
result = requests.get(url)
content = result.text

soup_hr5_115 = BeautifulSoup(content, 'lxml')

hr5_115_string = str(soup_hr5_115)

In [45]:
hr5_115_string

"<html><body><pre>\n[Congressional Bills 115th Congress]\n[From the U.S. Government Publishing Office]\n[H.R. 5 Referred in Senate (RFS)]\n\n&lt;DOC&gt;\n115th CONGRESS\n  1st Session\n                                 H. R. 5\n\n\n_______________________________________________________________________\n\n\n                   IN THE SENATE OF THE UNITED STATES\n\n                            January 12, 2017\n\nReceived; read twice and referred to the Committee on Homeland Security \n                        and Governmental Affairs\n\n_______________________________________________________________________\n\n                                 AN ACT\n\n\n \n To reform the process by which Federal agencies analyze and formulate \n   new regulations and guidance documents, to clarify the nature of \njudicial review of agency interpretations, to ensure complete analysis \n    of potential impacts on small entities of rules, and for other \n                               purposes.\n\n    Be it

In [42]:
soup_hr5_115_df = extract_sections(soup_hr5_115)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 soup_hr5_115_df = extract_sections(soup_hr5_115)                                             │
│   2                                                                                              │
│                                                                                                  │
│ in extract_sections:9                                                                            │
│                                                                                                  │
│    6 │   section_pattern = re.compile(r'SEC\.\s+(\d+)\.\s+(.*?)(?=\nSEC\.\s+\d+\.|$)', re.DOT    │
│    7 │                                                                                           │
│    8 │   # Find all matches in the input text                                                    │
│ ❱  9 │   matches = section_pattern.findall(input_text)                                           │
│   10 │                                                                                           │
│   11 │   # Create a DataFrame from the matches                                                   │
│   12 │   df = pd.DataFrame(matches, columns=['SectionNumber', 'SectionText'])                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: expected string or bytes-like object

In [46]:
import pandas as pd
import re

def extract_sections(input_text):
    # Define a case-insensitive regular expression pattern to identify sections
    section_pattern = re.compile(r'sec\.\s+(\d+)\.\s+(.*?)(?=\nsec\.\s+\d+\.|$)', re.DOTALL | re.IGNORECASE)

    # Find all matches in the input text
    matches = section_pattern.findall(input_text)

    # Create a DataFrame from the matches
    df = pd.DataFrame(matches, columns=['SectionNumber', 'SectionText'])
    df['SectionTitle'] = df['SectionText'].apply(lambda x: x.split('.')[0].strip())

    # Exclude the title from the SectionText column
    df['SectionTextWithoutTitle'] = df.apply(lambda row: row['SectionText'].replace(row['SectionTitle'], '').strip(), axis=1)

    return df




In [47]:
soup_hr5_115_df = extract_sections(soup_hr5_115)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 soup_hr5_115_df = extract_sections(soup_hr5_115)                                             │
│   2                                                                                              │
│                                                                                                  │
│ in extract_sections:9                                                                            │
│                                                                                                  │
│    6 │   section_pattern = re.compile(r'sec\.\s+(\d+)\.\s+(.*?)(?=\nsec\.\s+\d+\.|$)', re.DOT    │
│    7 │                                                                                           │
│    8 │   # Find all matches in the input text                                                    │
│ ❱  9 │   matches = section_pattern.findall(input_text)                                           │
│   10 │                                                                                           │
│   11 │   # Create a DataFrame from the matches                                                   │
│   12 │   df = pd.DataFrame(matches, columns=['SectionNumber', 'SectionText'])                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: expected string or bytes-like object

## use congress.gov api to get texts of enrolled bills?

In [ ]:
# manually gather list of numbers of endorsed Hour Resoultions from 113th congress 1st session
# use for loop to enter those HR numbers in query url for api call
# from json return gather most recent (first listed) link
# scrape text from link